# Preparing


## pip installs and import

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install scikit-learn

In [6]:
import json
import os

import torch
import torch.nn as nn
from tqdm import trange
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from datasets import load_metric
from sklearn.metrics import f1_score
import pandas as pd
import copy
import numpy as np

from transformers import ElectraModel, ElectraTokenizer
from transformers import AutoModel, ElectraTokenizer
from collections import Counter

import re

## git clone and file download

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/HappyBusDay/Korean_ABSA.git

In [ ]:
#Pt파일 다운로드
!gdown --id 1kThAuHqt2gN4Z2ULL6P7fOAOkjVQvDoT
!gdown --id 1URP7WC6xSTf9iP8NF3imY4TFnDPbqSQ0

# Config

In [9]:
PADDING_TOKEN = 1
S_OPEN_TOKEN = 0
S_CLOSE_TOKEN = 2

batch_size = 32

# pretrained_models
model_ELECTRA = 'kykim/electra-kor-base'
model_RoBERTa = 'xlm-roberta-base'
model_DeBERTa = "lighthouse/mdeberta-v3-base-kor-further"

entity_property_pair = [    # 카테고리의 수 = 25개
     '패키지/구성품#다양성','본품#인지도','브랜드#디자인',
     '패키지/구성품#편의성','제품 전체#디자인', '제품 전체#품질',
     '패키지/구성품#품질','패키지/구성품#일반','본품#일반',
     '패키지/구성품#디자인','본품#편의성','브랜드#품질',
     '브랜드#인지도','본품#다양성','본품#디자인',
     '제품 전체#다양성','본품#품질','제품 전체#인지도',
     '패키지/구성품#가격','본품#가격','제품 전체#가격',
     '브랜드#가격','브랜드#일반','제품 전체#일반','제품 전체#편의성'
     ]

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

special_tokens_dict = {
    'additional_special_tokens': ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
}

# functions and classes

## (func) jsonload, jsondump, jsonlload

In [10]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j

# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

## (func) tokenize_and_align_labels

In [11]:
def tokenize_and_align_labels(tokenizer, form, annotations, max_len):

    entity_property_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }
    polarity_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }

    for pair in entity_property_pair:
        isPairInOpinion = False
        if pd.isna(form):
            break
        tokenized_data = tokenizer(form, pair, padding='max_length', max_length=max_len, truncation=True)
        for annotation in annotations:
            entity_property = annotation[0]
            polarity = annotation[2]

            if polarity == '------------':
                continue

            if entity_property == pair:
                entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
                entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
                entity_property_data_dict['label'].append(tf_name_to_id['True'])

                polarity_data_dict['input_ids'].append(tokenized_data['input_ids'])
                polarity_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
                polarity_data_dict['label'].append(polarity_name_to_id[polarity])

                isPairInOpinion = True
                break

        if isPairInOpinion is False:
            entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
            entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
            entity_property_data_dict['label'].append(tf_name_to_id['False'])

    return entity_property_data_dict, polarity_data_dict

## (func) get_dataset

In [12]:
def get_dataset(raw_data, tokenizer, max_len):
    input_ids_list = []
    attention_mask_list = []
    token_labels_list = []

    polarity_input_ids_list = []
    polarity_attention_mask_list = []
    polarity_token_labels_list = []

    for utterance in raw_data:
        entity_property_data_dict, polarity_data_dict = tokenize_and_align_labels(tokenizer, utterance['sentence_form'], utterance['annotation'], max_len)
        
        input_ids_list.extend(entity_property_data_dict['input_ids'])
        attention_mask_list.extend(entity_property_data_dict['attention_mask'])
        token_labels_list.extend(entity_property_data_dict['label'])

        polarity_input_ids_list.extend(polarity_data_dict['input_ids'])
        polarity_attention_mask_list.extend(polarity_data_dict['attention_mask'])
        polarity_token_labels_list.extend(polarity_data_dict['label'])

    return TensorDataset(torch.tensor(input_ids_list), torch.tensor(attention_mask_list),
                         torch.tensor(token_labels_list)), TensorDataset(torch.tensor(polarity_input_ids_list), torch.tensor(polarity_attention_mask_list),
                         torch.tensor(polarity_token_labels_list))

## (class) SimpleClassifier

In [13]:
# baseline
class SimpleClassifier_Base(nn.Module):
    def __init__(self, num_label, classifier_hidden):
        super().__init__()

        if layer_use == True:
            self.dense1 = nn.Linear(classifier_hidden, classifier_hidden//2)
            self.dense2 = nn.Linear(classifier_hidden//2, classifier_hidden//4)
            self.dropout = nn.Dropout(dropout)
            self.output = nn.Linear(classifier_hidden//4, num_label)
        else:
            self.dense = nn.Linear(classifier_hidden, classifier_hidden)
            self.dropout = nn.Dropout(dropout)
            self.output = nn.Linear(classifier_hidden, num_label)

    def forward(self, features):
        if layer_use == True:
            x = features[:, 0, :]
            x = self.dropout(x) # layer 1
            x = self.dense1(x)
            x = self.dropout(x) # layer 2
            x = self.dense2(x)
            x = torch.tanh(x)
            x = self.dropout(x)
            x = self.output(x)
            return x

        else:
            x = features[:, 0, :]
            x = self.dropout(x)
            x = self.dense(x)
            x = torch.tanh(x)
            x = self.dropout(x)
            x = self.output(x)
            return x

## (class) BaseClassifier

* ELECTRA

In [14]:
class BaseClassifier_ELECTRA(nn.Module):
    def __init__(self, num_label, len_tokenizer, hidden_size):
        super(BaseClassifier_ELECTRA, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained( model_ELECTRA )
        self.electra.resize_token_embeddings(len_tokenizer)
        self.labels_classifier = SimpleClassifier_Base(self.num_label, hidden_size)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

* DeBERTa

In [15]:
class BaseClassifier_DeBERTa(nn.Module):
    def __init__(self, num_label, len_tokenizer, hidden_size):
        super(BaseClassifier_DeBERTa, self).__init__()

        self.num_label = num_label
        self.deberta = AutoModel.from_pretrained( model_DeBERTa )
        self.deberta.resize_token_embeddings(len_tokenizer)
        self.labels_classifier = SimpleClassifier_Base(self.num_label, hidden_size)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.deberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

* RoBERTa

In [16]:
class BaseClassifier_RoBERTa(nn.Module):
    def __init__(self, num_label, len_tokenizer, hidden_size):
        super(BaseClassifier_RoBERTa, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained( model_RoBERTa )
        self.electra.resize_token_embeddings(len_tokenizer)
        self.labels_classifier = SimpleClassifier_Base(self.num_label, hidden_size)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

* ELECTRA (polarity)

In [17]:
class BaseClassifier_Pola(nn.Module):
    def __init__(self, num_label, len_tokenizer, hidden_size):
        super(BaseClassifier_Pola, self).__init__()

        self.num_label = num_label
        self.electra = AutoModel.from_pretrained( model_ELECTRA )
        self.electra.resize_token_embeddings(len_tokenizer)
        self.labels_classifier = SimpleClassifier_Base(self.num_label, hidden_size)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

## (func) predict_from_korean_form

In [18]:
def predict_from_korean_form(tokenizer_cate, tokenizer_pola, ce_model, pc_model, data):
    category_tokenizer = tokenizer_cate
    polarity_tokenizer = tokenizer_pola

    ce_model.to(device)
    ce_model.eval()

    for idx, sentence in enumerate(data):
        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is arong: ", form)
            continue
            
        tmp= []
        force_flag = False

        # categoty(속성 범주)와 polarity(감성 범주) 모델을 각각 처리
        for pair in entity_property_pair: 
            tokenized_data_cate = category_tokenizer(form, pair, padding='max_length', max_length = max_length, truncation=True)
            tokenized_data_pola = polarity_tokenizer(form, pair, padding='max_length', max_length = polarity_max_length, truncation=True)
            input_ids_cate = torch.tensor([tokenized_data_cate['input_ids']]).to(device)
            input_ids_pola = torch.tensor([tokenized_data_pola['input_ids']]).to(device)
            attention_mask_cate = torch.tensor([tokenized_data_cate['attention_mask']]).to(device)
            attention_mask_pola = torch.tensor([tokenized_data_pola['attention_mask']]).to(device)

            with torch.no_grad():
                _, ce_logits = ce_model(input_ids_cate, attention_mask_cate)
                tmp.append( ce_logits[0][0] )
            
            # threshold 조정 시 ( default = 0 )
            if ce_logits[0][0] > threshold: 
                ce_predictions = torch.argmax(ce_logits, dim = -1)
                ce_result = tf_id_to_name[ce_predictions[0]]

                if ce_result == 'True':
                    force_flag = True
                    with torch.no_grad():
                        _, pc_logits = pc_model(input_ids_pola, attention_mask_pola)

                    pc_predictions = torch.argmax(pc_logits, dim=-1)
                    pc_result = polarity_id_to_name[pc_predictions[0]]

                    sentence['annotation'].append([pair, pc_result])

        # force evaluation of argument 사용 시 (default = False)
        if force_use == True: 
            if force_flag == False:
                tmp = torch.tensor(tmp)
                pair = entity_property_pair[torch.argmax(tmp)]
                with torch.no_grad():
                    _, pc_logits = pc_model(input_ids_pola, attention_mask_pola)

                pc_predictions = torch.argmax(pc_logits, dim=-1)
                pc_result = polarity_id_to_name[pc_predictions[0]]
                sentence['annotation'].append([pair, pc_result])

    return data

## (func) test_tentiment_analysis

In [19]:
def test_sentiment_analysis():
    print("model pt path  =", model_pt_path)
    print("polarity model pt path =", polarity_model_pt_path)

    if model_kind == 'RoBERTa':
        tokenizer = tokenizer_roberta
        model = BaseClassifier_RoBERTa(len(tf_id_to_name), len(tokenizer), hidden_size)

    elif model_kind == 'DeBERTa':
        tokenizer = tokenizer_deberta
        model = BaseClassifier_DeBERTa(len(tf_id_to_name), len(tokenizer), hidden_size)

    else :
        tokenizer = tokenizer_electra
        model = BaseClassifier_ELECTRA(len(tf_id_to_name), len(tokenizer), hidden_size)
        
    model.load_state_dict(torch.load(model_pt_path , map_location=device))
    model.to(device)
    model.eval()

    polarity_model = BaseClassifier_Pola(len(polarity_id_to_name), len(tokenizer_electra), polarity_hidden_size)
    polarity_model.load_state_dict(torch.load(polarity_model_pt_path , map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    pred_data = predict_from_korean_form(tokenizer, tokenizer_electra, model, polarity_model, copy.deepcopy(test_data))

    df_pred = pd.DataFrame(pred_data)

    with open(save_path  + '.jsonl', 'w') as file:
        for i in range( len(df_pred) ):
            tmp = str(df_pred['annotation'][i]).replace("\'", "\"").replace('None', 'null')
            file.write(  '{'+'\"id\": \"nikluge-sa-2022-test-{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                .format( str(i+1).zfill(5)  ,   df_pred['sentence_form'][i], tmp ) +'}' ) 
            file.write("\n")

    return df_pred

## (func) ensemble

In [ ]:
def excute_ensemble(model_list):
    dic_ae = {
        'id' : [],
        'sentence_form' : [],
        'annotation' : []
    }

    models = model_list 

    for i in range(len(models[0])):
        tmp_divide = []

        for j in models:
            tmp_divide.append( str(j['annotation'][i]) )

        answer = Counter(tmp_divide).most_common(n=1)[0][0]

        check =  str(tmp_divide).replace("[", "").replace("]", "").replace(",", "").replace("\'", "").replace("\"", "").replace(" ", "")

        if '[]' in answer and check is not "":
            while '[]' in tmp_divide:
                tmp_divide.remove('[]')
            dic_ae['annotation'].append( tmp_divide[0] )
        else:
            dic_ae['annotation'].append( answer )
        dic_ae['id'].append( j['id'][i] )
        dic_ae['sentence_form'].append( j['sentence_form'][i] )

    df_ae = pd.DataFrame( dic_ae )

    return df_ae

In [ ]:
def save_jsonl(df_ae, name, path):
    with open("{1}/{0}.jsonl" .format(name, path), 'w') as file:
        for i in range( len(df_ae) ):
            annos = df_ae['annotation'][i]
            str_annos = str(annos)
            tmp = str_annos.replace("None", "null").replace("\'", "\"")

            file.write(  '{'+'\"id\": \"nikluge-sa-2022-{3}-{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                        .format( str(i+1).zfill(5)  ,   df_ae['sentence_form'][i], tmp, name ) +'}' )
            file.write("\n")

# Starting Inference

parameter 설명
* model_kind (대소문자 구별)
    - RoBERTa : RoBERTa pretrained model load
    - DeBERTa : DeBERTa pretrained model load
    - ELECTRA : ELECTRA pretrained model load
* layer_use
    - True : add 1 layer
    - False : do not add layer (*default)
* hidden_size
    - defalut : 768 
    - up sampling : 1000
    - down sampling : 384
* force_use
    - True : force evalutation of argument
    - False : do not force evaluation (*default)
* threshold
    - default : 0
    - strict : 2 ~ 3
* dropout
    - default : 0.1

In [ ]:
############### 입력하세요 ###################

model_kind = 'ELECTRA' 

layer_use = False   # default = False
hidden_size = 768   # default = 768
force_use = False   # default = False
threshold = 0       # default = 0
dropout = 0.1       # default = 0.1

model_pt_path = "./G_Integral_kElectra_Lr_epoch_2.pt.pt의 사본"
polarity_model_pt_path = "./kelectra_polarity_epoch_58.pt.pt의 사본"

save_path = './test01'

###########################################

if model_kind == 'RoBERTa':
    max_length = 514
else:
    max_length = 256
polarity_max_length = 256
polarity_hidden_size = 768

* 1회만 실행

In [ ]:
tokenizer_roberta = AutoTokenizer.from_pretrained( model_RoBERTa )
tokenizer_electra = AutoTokenizer.from_pretrained( model_ELECTRA )
tokenizer_deberta = AutoTokenizer.from_pretrained( model_DeBERTa )

num_added_toks_roberta = tokenizer_roberta.add_special_tokens(special_tokens_dict)
num_added_toks_electra = tokenizer_electra.add_special_tokens(special_tokens_dict)
num_added_toks_deberta = tokenizer_deberta.add_special_tokens(special_tokens_dict)

test_data_path = './Korean_ABSA/data/test_data.jsonl'
test_data = jsonlload(test_data_path)

entity_property_test_data_roberta, polarity_test_data_roberta = get_dataset(test_data, tokenizer_roberta, max_length)  # max_length = 514
entity_property_test_data_electra, polarity_test_data_electra = get_dataset(test_data, tokenizer_electra, max_length)  # max_length = 256
entity_property_test_data_deberta, polarity_test_data_deberta = get_dataset(test_data, tokenizer_deberta, max_length)  # max_length = 256

entity_property_test_dataloader_roberta = DataLoader(entity_property_test_data_roberta, shuffle=True, batch_size=batch_size)
entity_property_test_dataloader_electra = DataLoader(entity_property_test_data_electra, shuffle=True, batch_size=batch_size)
entity_property_test_dataloader_deberta = DataLoader(entity_property_test_data_deberta, shuffle=True, batch_size=batch_size)

polarity_test_dataloader = DataLoader(polarity_test_data_electra, shuffle=True, batch_size=batch_size)

In [ ]:
test_sentiment_analysis()

#Starting Ensemble

## 단일모델


In [ ]:
######사용되는 단일모델#######

kelec_Gpu_6_LR2  = pd.DataFrame(jsonlload("./test01.jsonl"))

kelec_Gpu_19 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_19.jsonl"))
kelec_Gpu_16 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_16.jsonl"))
kelec_Gpu_11 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_11.jsonl"))
kelec_Gpu_15 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_15.jsonl"))
kelec_Gpu_20 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_20.jsonl"))
kelec_D_30        = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_D_30.jsonl"))
kelec_D_drop05_20 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_D_drop05_20.jsonl"))
roberta_Gpu_16  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/roberta_Gpu_16.jsonl"))
roberta_Gpu_15  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/roberta_Gpu_15.jsonl"))
deberta_Gpu_7   = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/deberta_Gpu_7.jsonl"))
deberta_Gpu_14  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/deberta_Gpu_14.jsonl"))
kelec_Gputrl_9  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gputrl_9.jsonl"))
kelec_Gputrl_11 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gputrl_11.jsonl"))
kelec_Gputrl_10 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gputrl_10.jsonl"))
kelec_Gputrl_7  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gputrl_7.jsonl"))
kelec_Gpu_6 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6.jsonl"))
deberta_Gpu_7F      = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/deberta_Gpu_7F.jsonl"))
deberta_Gpu_20F     = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/deberta_Gpu_20F.jsonl"))
kelec_Gpu_drop05_10 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_drop05_10.jsonl"))
kelec_Gpu_drop05_7  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_drop05_7.jsonl"))
kelec_Gpu_6F        = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6F.jsonl"))
kelec_Gpu_20F       = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_20F.jsonl"))
kelec_Gpu_itgr_LR2F = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_itgr_LR2F.jsonl"))
kelec_K_9           = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_K_9.jsonl"))
kelec_K_8           = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_K_8.jsonl"))
kelec_Gpu_itgr_LR2  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_itgr_LR2.jsonl"))
kelec_Gpu_itgr_LR1  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_itgr_LR1.jsonl"))
kelec_Gpu_itgr_drop05_9_layer = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_itgr_drop05_9_layer.jsonl"))
kelec_Gpu_itgr_drop05_8_layer = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_itgr_drop05_8_layer.jsonl"))
kelec_Gpu_6F_LR1 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6F_LR1.jsonl"))
kelec_Gpu_6F_LR2 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6F_LR2.jsonl"))
kelec_Gpu_6F_LR3 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6F_LR3.jsonl"))
kelec_Gpu_6_58   = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6_58.jsonl"))
kelec_K_drop_05_7_Ma32_LR2           = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_K_drop_05_7_Ma32_LR2.jsonl"))
kelec_K_drop_05_7_LR_1_Ma32_LR_2     = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_K_drop_05_7_LR_1_Ma32_LR_2.jsonl"))
kelec_K_drop_05_7_LR_1_Ma32_LR_2F    = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_K_drop_05_7_LR_1_Ma32_LR_2F.jsonl"))
kelec_Gpu_6_LR1_Ma32_LR_2F           = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6_LR1_Ma32_LR_2F.jsonl"))
kelec_Gpu_6_LR_1_step3more_Ma32_LR_2 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6_LR_1_step3more_Ma32_LR_2.jsonl"))
kelec_Gpu_6_LR_1_step3more_Ma32F     = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6_LR_1_step3more_Ma32F.jsonl"))
kelec_Gpu_LR6_attemp2_LR3            = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_LR6_attemp2_LR3.jsonl"))
kelec_Gpu_6_Ma32                     = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_6_Ma32.jsonl"))
kelec_Gpu_itgr_drop_layer_12         = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_Gpu_itgr_drop_layer_12.jsonl"))
kelec_K_drop_05_7                    = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_K_drop_05_7.jsonl"))
kelec_K_15            = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_K_15.jsonl"))
kelec_refind_8_Ma32 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_refind_8_Ma32.jsonl"))
kelec_refind_7_Ma32 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_refind_7_Ma32.jsonl"))
kelec_refind_6_Ma32 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_refind_6_Ma32.jsonl"))
kelec_refind_5_Ma32 = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kelec_refind_5_Ma32.jsonl"))
Deberta_Kdr_GPU_Kup_Original_Forcing  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/Deberta_Kdr_GPU_Kup_Original_Forcing.jsonl"))
Deberta_Kup_GPU_Kdr_Original_Forcing  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/Deberta_Kup_GPU_Kdr_Original_Forcing.jsonl"))
Original_Kup8_Gpu_Kdr_Deberta_Forcing = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/Original_Kup8_Gpu_Kdr_Deberta_Forcing.jsonl"))
Original_Kdr_Gpu_Kup8_Deberta_Forcing = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/Original_Kdr_Gpu_Kup8_Deberta_Forcing.jsonl"))
gpu_kdr_kup8_deberta_forcing = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/gpu_kdr_kup8_deberta_forcing.jsonl"))
kup8_gpu_kdr_deberta_forcing = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kup8_gpu_kdr_deberta_forcing.jsonl"))
GpuLR_175       = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/GpuLR_175.jsonl"))
Original_175     = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/Original_175.jsonl"))
Kup_175          = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/Kup_175.jsonl"))
K_175            = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/K_175.jsonl"))
deberta_Gpu_8    = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/deberta_Gpu_8.jsonl"))
roberta_Gpu_9    = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/roberta_Gpu_9.jsonl"))
kup_kdr_gpu_deberta_forcing = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/kup_kdr_gpu_deberta_forcing.jsonl"))
Original_8_Ma32   = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/Original_8_Ma32.jsonl"))
Original_9_Ma32   = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/Original_9_Ma32.jsonl"))
Original_10_Ma32  = pd.DataFrame(jsonlload("./Korean_ABSA/jsonl_files/Original_10_Ma32.jsonl"))

## 앙상블 모델

### kel5

In [ ]:
esb_1030_kel5_list  = [kelec_Gpu_19 ,  kelec_Gpu_16,  kelec_Gpu_11, kelec_Gpu_15,  kelec_Gpu_20]

name = 'esb_1030_kel5'
df_model = excute_ensemble(esb_1030_kel5_list)
save_jsonl(df_model, name, path)

esb_1030_kel5 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### kel7D

In [ ]:
esb_1030_kel7D_list = [kelec_Gpu_19 ,  kelec_Gpu_16,  kelec_Gpu_11, kelec_Gpu_15,  kelec_Gpu_20, kelec_D_30, kelec_D_drop05_20]

name = 'esb_1030_kel7D'
df_model = excute_ensemble(esb_1030_kel7D_list)
save_jsonl(df_model, name, path)

esb_1030_kel7D = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### many10s

In [ ]:
esb_1030_many10s_list = [kelec_Gpu_19 ,  kelec_Gpu_16, roberta_Gpu_16, roberta_Gpu_15, 
                   deberta_Gpu_7, deberta_Gpu_14, kelec_Gputrl_9, kelec_Gputrl_11, kelec_D_30, kelec_D_drop05_20]

name = 'esb_1030_many10s'
df_model = excute_ensemble(esb_1030_many10s_list)
save_jsonl(df_model, name, path)

esb_1030_many10s = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### kelGpu_Gputgr8

In [ ]:
esb_1030_kelGpu_Gputgr8_list = [kelec_Gpu_19 ,  kelec_Gpu_16,  kelec_Gpu_11, kelec_Gpu_15,
                                kelec_Gputrl_9, kelec_Gputrl_11, kelec_Gputrl_10, kelec_Gputrl_7]

name = 'esb_1030_kelGpu_Gputgr8'
df_model = excute_ensemble(esb_1030_kelGpu_Gputgr8_list)
save_jsonl(df_model, name, path)

esb_1030_kelGpu_Gputgr8 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### max 23_3

In [ ]:
esb_1101_max23_3_list = [esb_1030_kelGpu_Gputgr8, esb_1030_kel7D, esb_1030_many10s,
                    kelec_Gpu_6, kelec_Gpu_11, kelec_D_30, kelec_D_drop05_20, kelec_Gputrl_9, kelec_Gputrl_11,
                    deberta_Gpu_7F, deberta_Gpu_20F, kelec_Gpu_drop05_10, kelec_Gpu_drop05_7,
                    kelec_Gpu_6F, kelec_Gpu_20F, kelec_Gpu_itgr_LR2F, kelec_K_9, kelec_K_8,
                    kelec_Gpu_itgr_LR2, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer, kelec_Gpu_itgr_drop05_8_layer]

name = 'esb_1101_max23_3'
df_model = excute_ensemble(esb_1101_max23_3_list)
save_jsonl(df_model, name, path)

esb_1101_max23_3 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### max 23_2

In [ ]:
esb_1101_max23_2_list = [esb_1030_kelGpu_Gputgr8, esb_1030_kel5, esb_1030_many10s,
                        kelec_Gpu_6, kelec_Gpu_11, kelec_D_30, kelec_D_drop05_20, kelec_Gputrl_9, kelec_Gputrl_11,
                        deberta_Gpu_7F, deberta_Gpu_20F, kelec_Gpu_drop05_10, kelec_Gpu_drop05_7,
                        kelec_Gpu_6F, kelec_Gpu_20F, kelec_Gpu_itgr_LR2F, kelec_K_9, kelec_K_8,
                        kelec_Gpu_itgr_LR2, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer, kelec_Gpu_itgr_drop05_8_layer]

                    
name = 'esb_1101_max23_2'
df_model = excute_ensemble(esb_1101_max23_2_list)
save_jsonl(df_model, name, path)

esb_1101_max23_2 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### max 27_13

In [ ]:
esb_1102_max27_13_list = [kelec_Gpu_6F_LR1, kelec_Gpu_6F_LR2, kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                     esb_1030_kel5, esb_1030_many10s, kelec_Gpu_6_58, kelec_Gpu_6, kelec_Gpu_11,
                     kelec_D_30, kelec_D_drop05_20, kelec_Gputrl_11, deberta_Gpu_7F,
                     deberta_Gpu_20F, kelec_Gpu_drop05_10, kelec_Gpu_drop05_7, kelec_Gpu_6F, kelec_Gpu_20F,
                     kelec_Gpu_itgr_LR2F, kelec_K_9, kelec_K_8, kelec_Gpu_itgr_LR2, kelec_Gpu_itgr_LR1,
                     kelec_Gpu_itgr_drop05_9_layer, kelec_Gpu_itgr_drop05_8_layer]


name = 'esb_1102_max27_13'
df_model = excute_ensemble(esb_1102_max27_13_list)
save_jsonl(df_model, name, path)

esb_1102_max27_13 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### max 27_6

In [ ]:
esb_1102_max27_6_list = [kelec_Gpu_6F_LR1, kelec_Gpu_6F_LR2, kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        esb_1030_many10s, kelec_Gpu_6_58, kelec_Gpu_6, kelec_Gpu_11,
                        kelec_D_30, kelec_D_drop05_20, kelec_Gputrl_9, kelec_Gputrl_11, deberta_Gpu_7F,
                        deberta_Gpu_20F, kelec_Gpu_drop05_10, kelec_Gpu_drop05_7, kelec_Gpu_6F, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_9, kelec_K_8, kelec_Gpu_itgr_LR2, kelec_Gpu_itgr_LR1,
                        kelec_Gpu_itgr_drop05_9_layer, kelec_Gpu_itgr_drop05_8_layer]

name = 'esb_1102_max27_6'
df_model = excute_ensemble(esb_1102_max27_6_list)
save_jsonl(df_model, name, path)

esb_1102_max27_6 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### max 26_x2

In [ ]:
esb_1101_max26_x2_list = [esb_1030_kelGpu_Gputgr8, esb_1030_kelGpu_Gputgr8, esb_1030_kel7D, esb_1030_kel7D, esb_1030_kel5, esb_1030_many10s,
                        kelec_Gpu_6, kelec_Gpu_11, kelec_D_30, kelec_D_drop05_20, kelec_Gputrl_9, kelec_Gputrl_11,
                        deberta_Gpu_7F, deberta_Gpu_20F, kelec_Gpu_drop05_10, kelec_Gpu_drop05_7,
                        kelec_Gpu_6F, kelec_Gpu_20F, kelec_Gpu_itgr_LR2F, kelec_K_9, kelec_K_8,
                        kelec_Gpu_itgr_LR2, kelec_Gpu_itgr_LR2, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer, kelec_Gpu_itgr_drop05_8_layer]

name = 'esb_1101_max26_x2'
df_model = excute_ensemble(esb_1101_max26_x2_list)
save_jsonl(df_model, name, path)

esb_1101_max26_x2 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### desire 36_18

In [ ]:
esb_1103_desire36_18_list = [esb_1101_max23_2, esb_1101_max23_3, esb_1101_max26_x2, 
                    kelec_Gpu_6F_LR1, kelec_Gpu_6F_LR2, 
                    kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                    kelec_K_drop_05_7_LR_1_Ma32_LR_2F , kelec_Gpu_6_LR1_Ma32_LR_2F ,
                    kelec_Gpu_6_LR_1_step3more_Ma32_LR_2 , kelec_Gpu_6_LR_1_step3more_Ma32F ,
                    kelec_Gpu_LR6_attemp2_LR3 , kelec_Gpu_6_Ma32 ,kelec_Gpu_itgr_drop_layer_12, kelec_K_drop_05_7  ,
                    kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, 
                    esb_1030_many10s, kelec_Gpu_6_58, kelec_Gpu_6, kelec_Gpu_11,
                    kelec_D_30, kelec_D_drop05_20, 
                    deberta_Gpu_20F, kelec_Gpu_drop05_10, kelec_Gpu_drop05_7, kelec_Gpu_6F, kelec_Gpu_20F,
                    kelec_Gpu_itgr_LR2F, kelec_K_9, kelec_K_8, kelec_Gpu_itgr_LR2, kelec_Gpu_itgr_LR1,
                    kelec_Gpu_itgr_drop05_9_layer, kelec_Gpu_itgr_drop05_8_layer]

name = 'esb_1103_desire36_18'
df_model = excute_ensemble(esb_1103_desire36_18_list)
save_jsonl(df_model, name, path)

esb_1103_desire36_18 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### max D 3_2

In [ ]:
esb_1103_maxD3_2_list = [esb_1101_max23_2,  esb_1101_max26_x2,
                        kelec_Gpu_6F_LR1, kelec_Gpu_6F_LR2, 
                        kelec_Gpu_LR6_attemp2_LR3 , kelec_Gpu_6_Ma32 ,kelec_Gpu_itgr_drop_layer_12, kelec_K_drop_05_7  ,
                        kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        esb_1030_many10s, kelec_Gpu_6_58, kelec_Gpu_6, kelec_Gpu_11,
                        kelec_D_30, kelec_D_drop05_20, 
                        deberta_Gpu_20F, kelec_Gpu_drop05_10, kelec_Gpu_drop05_7, kelec_Gpu_6F, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_9, kelec_K_8, kelec_Gpu_itgr_LR2, kelec_Gpu_itgr_LR1,
                        kelec_Gpu_itgr_drop05_9_layer, kelec_Gpu_itgr_drop05_8_layer]
                 
name = 'esb_1103_maxD3_2'
df_model = excute_ensemble(esb_1103_maxD3_2_list)
save_jsonl(df_model, name, path)

esb_1103_maxD3_2 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### hh 30_14

In [ ]:
esb_1103_hh30_14_list = [esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3, 
                        kelec_Gpu_6F_LR1, kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                        kelec_K_drop_05_7_LR_1_Ma32_LR_2F , kelec_Gpu_6_LR1_Ma32_LR_2F ,
                        kelec_Gpu_6_LR_1_step3more_Ma32_LR_2 , kelec_Gpu_6_LR_1_step3more_Ma32F ,
                        kelec_Gpu_LR6_attemp2_LR3 , kelec_Gpu_6_Ma32 ,kelec_K_drop_05_7  ,
                        kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        esb_1030_many10s, kelec_Gpu_6_58, kelec_Gpu_6, 
                        kelec_D_30, kelec_D_drop05_20, 
                        deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1103_hh30_14'
df_model = excute_ensemble(esb_1103_hh30_14_list)
save_jsonl(df_model, name, path)

esb_1103_hh30_14 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### hh 30_5

In [ ]:
esb_1103_hh30_5_list = [esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3, 
                        kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                        kelec_K_drop_05_7_LR_1_Ma32_LR_2F , kelec_Gpu_6_LR1_Ma32_LR_2F ,
                        kelec_Gpu_6_LR_1_step3more_Ma32_LR_2 , kelec_Gpu_6_LR_1_step3more_Ma32F ,
                        kelec_Gpu_LR6_attemp2_LR3 , kelec_Gpu_6_Ma32 ,kelec_Gpu_itgr_drop_layer_12, kelec_K_drop_05_7  ,
                        kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        esb_1030_many10s, kelec_Gpu_6_58, kelec_Gpu_6, 
                        kelec_D_30, kelec_D_drop05_20, 
                        deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1103_hh30_5'
df_model = excute_ensemble(esb_1103_hh30_5_list)
save_jsonl(df_model, name, path)

esb_1103_hh30_5 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### hh 30_8

In [ ]:
esb_1103_hh30_8_list = [esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3, 
                        kelec_Gpu_6F_LR1, kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                        kelec_Gpu_6_LR1_Ma32_LR_2F ,
                        kelec_Gpu_6_LR_1_step3more_Ma32_LR_2 , kelec_Gpu_6_LR_1_step3more_Ma32F ,
                        kelec_Gpu_LR6_attemp2_LR3 , kelec_Gpu_6_Ma32 ,kelec_Gpu_itgr_drop_layer_12, kelec_K_drop_05_7  ,
                        kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        esb_1030_many10s, kelec_Gpu_6_58, kelec_Gpu_6, 
                        kelec_D_30, kelec_D_drop05_20, 
                        deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1103_hh30_8'
df_model = excute_ensemble(esb_1103_hh30_8_list)
save_jsonl(df_model, name, path)

esb_1103_hh30_8 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### refine4

In [ ]:
esb_1107_refine4_list = [kelec_refind_8_Ma32, kelec_refind_7_Ma32, kelec_refind_6_Ma32, kelec_refind_5_Ma32]

name = 'esb_1107_refine4'
df_model = excute_ensemble(esb_1107_refine4_list)
save_jsonl(df_model, name, path)

esb_1107_refine4 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### stroke4deb

In [ ]:
esb_1107_stroke4deb_list = [Deberta_Kdr_GPU_Kup_Original_Forcing, Deberta_Kup_GPU_Kdr_Original_Forcing,
                            Original_Kup8_Gpu_Kdr_Deberta_Forcing, Original_Kdr_Gpu_Kup8_Deberta_Forcing]
                       
name = 'esb_1107_stroke4deb'
df_model = excute_ensemble(esb_1107_stroke4deb_list)
save_jsonl(df_model, name, path)

esb_1107_stroke4deb = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### stroke6


In [ ]:
esb_1107_stroke6_list = [Deberta_Kdr_GPU_Kup_Original_Forcing, Deberta_Kup_GPU_Kdr_Original_Forcing,
                        Original_Kup8_Gpu_Kdr_Deberta_Forcing, Original_Kdr_Gpu_Kup8_Deberta_Forcing,
                        gpu_kdr_kup8_deberta_forcing ,kup8_gpu_kdr_deberta_forcing ]
                    
name = 'esb_1107_stroke6'
df_model = excute_ensemble(esb_1107_stroke6_list)
save_jsonl(df_model, name, path)

esb_1107_stroke6 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### force threshold 1.75 4


In [ ]:
esb_175_4_list = [GpuLR_175, Original_175,  Kup_175,  K_175]

name = 'esb_175_4'
df_model = excute_ensemble(esb_175_4_list)
save_jsonl(df_model, name, path)

esb_175_4 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### YD 32_23

In [ ]:
esb_1103_YD32_23_list = [esb_1103_hh30_8, esb_1103_hh30_14, esb_1103_hh30_5, esb_1103_maxD3_2, esb_1103_desire36_18,
                        esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3,  
                        kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2, kelec_Gpu_6_LR1_Ma32_LR_2F ,
                        kelec_Gpu_6_LR_1_step3more_Ma32_LR_2 , kelec_Gpu_6_LR_1_step3more_Ma32F ,
                        kelec_Gpu_LR6_attemp2_LR3 , kelec_Gpu_6_Ma32 ,kelec_K_drop_05_7  ,
                        kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        esb_1030_many10s, kelec_Gpu_6_58,  
                        kelec_D_30, kelec_D_drop05_20, 
                        deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1103_YD32_23'
df_model = excute_ensemble(esb_1103_YD32_23_list)
save_jsonl(df_model, name, path)

esb_1103_YD32_23 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### YD 32_16

In [ ]:
esb_1103_YD32_16_list = [esb_1103_hh30_8, esb_1103_hh30_14, esb_1103_hh30_5, esb_1103_maxD3_2, esb_1103_desire36_18,
                        esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3,  
                        kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2, kelec_Gpu_6_LR1_Ma32_LR_2F , 
                        kelec_Gpu_6_LR_1_step3more_Ma32_LR_2 , kelec_Gpu_6_LR_1_step3more_Ma32F ,
                        kelec_Gpu_LR6_attemp2_LR3 , kelec_K_drop_05_7  ,
                        kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        esb_1030_many10s, kelec_Gpu_6_58, kelec_Gpu_6, 
                        kelec_D_30, kelec_D_drop05_20, 
                        deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1103_YD32_16'
df_model = excute_ensemble(esb_1103_YD32_16_list)
save_jsonl(df_model, name, path)

esb_1103_YD32_16 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### GB 30_2

In [ ]:
esb_1103_GB30_2_list = [esb_1103_hh30_8,  esb_1103_hh30_5, esb_1103_maxD3_2, esb_1103_desire36_18,
                        esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3, 
                        kelec_Gpu_6F_LR1, kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                        kelec_Gpu_6_LR1_Ma32_LR_2F ,
                        kelec_Gpu_6_LR_1_step3more_Ma32_LR_2, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        kelec_Gpu_6_Ma32 , kelec_K_drop_05_7, esb_1030_many10s, kelec_Gpu_6_58,  kelec_Gpu_6_LR_1_step3more_Ma32F ,
                        kelec_D_30, kelec_D_drop05_20, deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1103_GB30_2'
df_model = excute_ensemble(esb_1103_GB30_2_list)
save_jsonl(df_model, name, path)

esb_1103_GB30_2 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### GB 30_3

In [ ]:
esb_1103_GB30_3_list = [esb_1103_hh30_8, esb_1103_hh30_14, esb_1103_maxD3_2, esb_1103_desire36_18,
                        esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3, 
                        kelec_Gpu_6F_LR1, kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                        kelec_Gpu_6_LR1_Ma32_LR_2F ,
                        kelec_Gpu_6_LR_1_step3more_Ma32_LR_2, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        kelec_Gpu_6_Ma32 , kelec_K_drop_05_7, esb_1030_many10s, kelec_Gpu_6_58,  kelec_Gpu_6_LR_1_step3more_Ma32F ,
                        kelec_D_30, kelec_D_drop05_20, deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1103_GB30_3'
df_model = excute_ensemble(esb_1103_GB30_3_list)
save_jsonl(df_model, name, path)

esb_1103_GB30_3 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### GB 30_15

In [ ]:
esb_1103_GB30_15_list = [esb_1103_hh30_8, esb_1103_hh30_14, esb_1103_hh30_5, esb_1103_maxD3_2, esb_1103_desire36_18,
                        esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3, 
                        kelec_Gpu_6F_LR1, kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                        kelec_Gpu_6_LR1_Ma32_LR_2F ,
                        kelec_Gpu_6_LR_1_step3more_Ma32_LR_2, esb_1030_kelGpu_Gputgr8,
                        kelec_Gpu_6_Ma32 , kelec_K_drop_05_7, esb_1030_many10s, kelec_Gpu_6_58,  kelec_Gpu_6_LR_1_step3more_Ma32F ,
                        kelec_D_30, kelec_D_drop05_20, deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1103_GB30_15'
df_model = excute_ensemble(esb_1103_GB30_15_list)
save_jsonl(df_model, name, path)

esb_1103_GB30_15 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### OB33_6

In [ ]:
esb_1106_OB33_6_list = [esb_1103_GB30_15, esb_1103_GB30_2, esb_1103_GB30_3, esb_1103_YD32_16, esb_1103_YD32_23,
                        esb_1103_hh30_8,  esb_1103_hh30_5, esb_1103_maxD3_2, esb_1103_desire36_18,
                        esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3, 
                        kelec_K_drop_05_7_Ma32_LR2 , kelec_Gpu_6_LR1_Ma32_LR_2F , roberta_Gpu_16,
                        kelec_Gpu_6_LR_1_step3more_Ma32_LR_2, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                        kelec_K_drop_05_7, esb_1030_many10s, kelec_Gpu_6_58,  deberta_Gpu_8, roberta_Gpu_9,
                        kelec_D_30, kelec_D_drop05_20, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                        kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_drop05_9_layer]

                   
name = 'esb_1106_OB33_6'
df_model = excute_ensemble(esb_1106_OB33_6_list)
save_jsonl(df_model, name, path)

esb_1106_OB33_6 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### X_MAS 28_12

In [ ]:
esb_1105_X_MAS28_12_list = [esb_1103_hh30_8, esb_1103_hh30_14, esb_1103_hh30_5, esb_1103_maxD3_2, esb_1103_desire36_18,
                            esb_1102_max27_6, esb_1102_max27_13, esb_1101_max23_2, esb_1101_max23_3, 
                            kelec_K_drop_05_7_Ma32_LR2 , kelec_Gpu_6_LR1_Ma32_LR_2F ,  roberta_Gpu_16,
                            kelec_Gpu_6_LR_1_step3more_Ma32_LR_2, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                            kelec_K_drop_05_7, esb_1030_many10s, kelec_Gpu_6_58,  deberta_Gpu_8, roberta_Gpu_9,
                            kelec_D_30, kelec_D_drop05_20, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                            kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1105_X_MAS28_12'
df_model = excute_ensemble(esb_1105_X_MAS28_12_list)
save_jsonl(df_model, name, path)

esb_1105_X_MAS28_12 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### FULLSTACK 33_2

In [ ]:
esb_1106_FULLSTACK33_2_list = [esb_1103_GB30_15 ,  esb_1103_YD32_16, esb_1103_YD32_23,
                                esb_1105_X_MAS28_12, kup_kdr_gpu_deberta_forcing,
                                esb_1103_hh30_8, esb_1103_hh30_14, esb_1103_maxD3_2, esb_1103_desire36_18,
                                esb_1102_max27_6, esb_1101_max23_2,   
                                kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2, kelec_Gpu_6_LR1_Ma32_LR_2F ,
                                kelec_Gpu_6_LR_1_step3more_Ma32_LR_2 , kelec_Gpu_6_LR_1_step3more_Ma32F ,
                                kelec_Gpu_LR6_attemp2_LR3 , kelec_K_drop_05_7  ,
                                kelec_Gpu_6F_LR3, kelec_Gpu_6_LR2, esb_1030_kelGpu_Gputgr8,
                                esb_1030_many10s, kelec_Gpu_6_58,  
                                kelec_D_30, kelec_D_drop05_20, 
                                deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                                kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1106_FULLSTACK33_2'
df_model = excute_ensemble(esb_1106_FULLSTACK33_2_list)
save_jsonl(df_model, name, path)

esb_1106_FULLSTACK33_2 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### NUCLEAR 30_4

In [ ]:
esb_1106_NUCLEAR30_4_list = [esb_1103_GB30_15, esb_1103_GB30_2, esb_1103_YD32_16, 
                            esb_1103_hh30_8, esb_1103_hh30_14,  esb_1103_maxD3_2, esb_1103_desire36_18,
                            esb_1102_max27_6, esb_1101_max23_2, 
                            kelec_Gpu_6F_LR1, kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                            kelec_Gpu_6_LR1_Ma32_LR_2F ,
                            kelec_Gpu_6_LR_1_step3more_Ma32_LR_2, esb_1030_kelGpu_Gputgr8,
                            kelec_Gpu_6_Ma32 , kelec_K_drop_05_7, esb_1030_many10s, kelec_Gpu_6_58,  kelec_Gpu_6_LR_1_step3more_Ma32F ,
                            kelec_D_30, kelec_D_drop05_20, deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                            kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1106_NUCLEAR30_4'
df_model = excute_ensemble(esb_1106_NUCLEAR30_4_list)
save_jsonl(df_model, name, path)

esb_1106_NUCLEAR30_4 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### terarosa 39_2

In [ ]:
esb_1107_terarosa39_2_list = [esb_1106_NUCLEAR30_4,  esb_1106_FULLSTACK33_2, esb_1106_OB33_6,
                            gpu_kdr_kup8_deberta_forcing, kup8_gpu_kdr_deberta_forcing, Original_Kup8_Gpu_Kdr_Deberta_Forcing,
                            esb_1107_refine4, kelec_refind_8_Ma32, kelec_refind_7_Ma32, kelec_refind_6_Ma32,
                            esb_1103_GB30_15, esb_1103_GB30_2, esb_1103_YD32_16, 
                            esb_1103_hh30_8,  esb_1103_maxD3_2, esb_1103_desire36_18,
                            esb_1102_max27_6, esb_1101_max23_2, 
                            kelec_Gpu_6F_LR1, kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                            kelec_Gpu_6_LR1_Ma32_LR_2F ,
                            kelec_Gpu_6_LR_1_step3more_Ma32_LR_2, esb_1030_kelGpu_Gputgr8,
                            kelec_Gpu_6_Ma32 , kelec_K_drop_05_7, esb_1030_many10s, kelec_Gpu_6_58,  kelec_Gpu_6_LR_1_step3more_Ma32F ,
                            kelec_D_30, kelec_D_drop05_20, deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                            kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1107_terarosa39_2'
df_model = excute_ensemble(esb_1107_terarosa39_2_list)
save_jsonl(df_model, name, path)

esb_1107_terarosa39_2 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### icon 36_11

In [ ]:
esb_1107_icon36_11_list = [esb_1106_NUCLEAR30_4, esb_1106_FULLSTACK33_2, esb_1106_OB33_6,       
                            esb_1107_stroke6, esb_1107_stroke4deb, Deberta_Kdr_GPU_Kup_Original_Forcing, 
                            Original_Kup8_Gpu_Kdr_Deberta_Forcing, 
                            esb_1107_refine4, kelec_refind_8_Ma32, kelec_refind_7_Ma32,
                            esb_1103_YD32_16, 
                            esb_1103_hh30_8,  esb_1103_maxD3_2, esb_1103_desire36_18,
                            esb_1102_max27_6,  
                            kelec_Gpu_6F_LR1, kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                            kelec_Gpu_6_LR1_Ma32_LR_2F ,
                            kelec_Gpu_6_LR_1_step3more_Ma32_LR_2, esb_1030_kelGpu_Gputgr8,
                            kelec_Gpu_6_Ma32 , kelec_K_drop_05_7, esb_1030_many10s, kelec_Gpu_6_58,  kelec_Gpu_6_LR_1_step3more_Ma32F ,
                            kelec_D_30, kelec_D_drop05_20, deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                            kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1107_icon36_11'
df_model = excute_ensemble(esb_1107_icon36_11_list)
save_jsonl(df_model, name, path)

esb_1107_icon36_11 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

### joy of victory 45_8

In [ ]:
esb_1108_joyofvictory45_8_list = [esb_1107_icon36_11, esb_1107_terarosa39_2, 
                                esb_175_4, GpuLR_175, Original_175,
                                esb_1106_NUCLEAR30_4,  esb_1106_FULLSTACK33_2, 
                                Original_8_Ma32, Original_9_Ma32, Original_10_Ma32,
                                esb_1106_NUCLEAR30_4, esb_1106_FULLSTACK33_2, esb_1106_OB33_6,
                                esb_1107_stroke6, esb_1107_stroke4deb, Deberta_Kdr_GPU_Kup_Original_Forcing, 
                                Original_Kup8_Gpu_Kdr_Deberta_Forcing, 
                                esb_1107_refine4, kelec_refind_8_Ma32, kelec_refind_7_Ma32,
                                esb_1103_hh30_8,  esb_1103_maxD3_2, esb_1103_desire36_18,
                                esb_1102_max27_6,  
                                kelec_Gpu_6F_LR1, kelec_K_drop_05_7_Ma32_LR2 , kelec_K_drop_05_7_LR_1_Ma32_LR_2 ,
                                kelec_Gpu_6_LR1_Ma32_LR_2F ,
                                kelec_Gpu_6_LR_1_step3more_Ma32_LR_2, esb_1030_kelGpu_Gputgr8,
                                kelec_Gpu_6_Ma32 , kelec_K_drop_05_7, esb_1030_many10s, kelec_Gpu_6_58,  kelec_Gpu_6_LR_1_step3more_Ma32F ,
                                kelec_D_30, kelec_D_drop05_20, deberta_Gpu_20F, kelec_Gpu_drop05_7, kelec_Gpu_20F,
                                kelec_Gpu_itgr_LR2F, kelec_K_15, kelec_Gpu_itgr_LR1, kelec_Gpu_itgr_drop05_9_layer]

name = 'esb_1108_joyofvictory45_8'
df_model = excute_ensemble(esb_1108_joyofvictory45_8_list)
save_jsonl(df_model, name, path)

esb_1108_joyofvictory45_8 = pd.DataFrame(jsonlload(path + "/" + name + ".jsonl"))

##앙상블 결과

In [ ]:
esb_1108_joyofvictory45_8